In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
# Import data retrieved from HookTheory API
hook_data = pd.read_csv('chords_all_songs.csv')
hook_data = hook_data.drop(columns='Unnamed: 0')
hook_data.head()

,artist,cp,section,song,type
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord
1,3 Doors Down,"1,5,6",Verse,Be Like That,3 chord
2,Adele,"1,5,6",Chorus,Someone Like You,3 chord
3,Aerosmith,"1,5,6",Verse,Cryin',3 chord
4,Ai Ootsuka,"1,5,6",Chorus,Sakuranbo,3 chord


In [3]:
# Display size of dataframe
len(hook_data)

48935

In [4]:
# Import data retrieved from last.fm and spotify for unique songs subset of HookTheory data
stats = pd.read_csv('spotify_data.csv')
stats.head()

,0,cp,section,song,type,song (last.fm),artist (last.fm),listener count (last.fm),play count (last.fm),genre (last.fm),song (Spotify),artist (Spotify),popularity (Spotify),release date (Spotify)
0,73113243,"1,2,3",Chorus,NYAN CAT,3 chord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10cc,"4,5,6",Intro and Verse,The Wall Street Shuffle,3 chord,The Wall Street Shuffle,10cc,66115.0,210783.0,"rock, classic rock, 70s, 10cc, pop",The Wall Street Shuffle,10cc,49.0,2004-01-01
2,1927,"1,4,5",Intro and Verse,If I Could,3 chord,If I Could,1927,4020.0,21222.0,"australian, pop, rock, 80s, AOR",NaN,NaN,NaN,NaN
3,2pac,"4,1,5",Verse,Changes,3 chord,Changes,2Pac,315181.0,2046155.0,"rap, Hip-Hop, hip hop, 2pac, Gangsta Rap",Changes,2Pac,64.0,1998-01-01
4,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,Be Like That,3 Doors Down,403366.0,1913091.0,"rock, alternative rock, 3 Doors Down, alternat...",Be Like That,3 Doors Down,60.0,2000


In [5]:
# Rename first column as 'artist'
stats = stats.rename(columns={'0':'artist'})

# Create a truncated dataframe
lastfm_raw = stats[['artist',
                       'song',
                       'listener count (last.fm)',
                       'play count (last.fm)',
                       'genre (last.fm)']]

# Drop null entries from dataframe
lastfm_raw = lastfm_raw.dropna(how='any')
lastfm_raw.head()

,artist,song,listener count (last.fm),play count (last.fm),genre (last.fm)
1,10cc,The Wall Street Shuffle,66115.0,210783.0,"rock, classic rock, 70s, 10cc, pop"
2,1927,If I Could,4020.0,21222.0,"australian, pop, rock, 80s, AOR"
3,2pac,Changes,315181.0,2046155.0,"rap, Hip-Hop, hip hop, 2pac, Gangsta Rap"
4,3 Doors Down,Be Like That,403366.0,1913091.0,"rock, alternative rock, 3 Doors Down, alternat..."
5,3 Doors Down,Kryptonite,1067856.0,7265039.0,"rock, alternative rock, 3 Doors Down, alternat..."


In [6]:
# Separate genre data for each song into individual rows
lastfm_genre = pd.DataFrame(lastfm_raw['genre (last.fm)'].str.split(', ').tolist(),index=lastfm_raw['song']).stack()
lastfm_genre

song                          
The Wall Street Shuffle      0                rock
                             1        classic rock
                             2                 70s
                             3                10cc
                             4                 pop
If I Could                   0          australian
                             1                 pop
                             2                rock
                             3                 80s
                             4                 AOR
Changes                      0                 rap
                             1             Hip-Hop
                             2             hip hop
                             3                2pac
                             4         Gangsta Rap
Be Like That                 0                rock
                             1    alternative rock
                             2        3 Doors Down
                             3         alternative


In [7]:
# Reset index to resulting dataframe
lastfm_genre = lastfm_genre.reset_index([0,'song'])

# Assign column headers
lastfm_genre.columns = ['song','genre (last.fm)']
lastfm_genre.head()

,song,genre (last.fm)
0,The Wall Street Shuffle,rock
1,The Wall Street Shuffle,classic rock
2,The Wall Street Shuffle,70s
3,The Wall Street Shuffle,10cc
4,The Wall Street Shuffle,pop


In [8]:
# Drop original genre column from original dataframe and merge with new dataframe
lastfm_raw = lastfm_raw.drop(columns=['genre (last.fm)'])
lastfm_stats = pd.merge(lastfm_raw, lastfm_genre, how='outer', on='song')
lastfm_stats.head()

,artist,song,listener count (last.fm),play count (last.fm),genre (last.fm)
0,10cc,The Wall Street Shuffle,66115.0,210783.0,rock
1,10cc,The Wall Street Shuffle,66115.0,210783.0,classic rock
2,10cc,The Wall Street Shuffle,66115.0,210783.0,70s
3,10cc,The Wall Street Shuffle,66115.0,210783.0,10cc
4,10cc,The Wall Street Shuffle,66115.0,210783.0,pop


In [9]:
# Determine if the values for artist and genre for each row match
# if lastfm_stats.loc[3,'artist'] == lastfm_stats.loc[3,'genre (last.fm)']:
#     print('True')
# else:
#     print('False')

In [10]:
# Inititalize list of index information for rows to be dropped from dataframe
droplist = []

# Use for loop to determine if either the artist name or the song name matches the genre for each row
# If either condition is met, add index number to droplist list
for song in range(len(lastfm_stats)):
    artist = lastfm_stats.loc[song,'artist']
    track = lastfm_stats.loc[song,'song']
    genre = lastfm_stats.loc[song,'genre (last.fm)']
    if (artist == genre) | (track == genre):
        droplist.append(song)

# Drop dataframe entries based on index information contained in droplist list
lastfm_stats.drop(droplist,inplace=True)
lastfm_stats.head()

,artist,song,listener count (last.fm),play count (last.fm),genre (last.fm)
0,10cc,The Wall Street Shuffle,66115.0,210783.0,rock
1,10cc,The Wall Street Shuffle,66115.0,210783.0,classic rock
2,10cc,The Wall Street Shuffle,66115.0,210783.0,70s
4,10cc,The Wall Street Shuffle,66115.0,210783.0,pop
5,1927,If I Could,4020.0,21222.0,australian


In [11]:
# Create a truncated dataframe
spotify_stats = stats[['artist',
                       'song',
                       'popularity (Spotify)',
                       'release date (Spotify)']]

# Drop null entries from dataframe
spotify_stats = spotify_stats.dropna(how='any')
spotify_stats.head()

,artist,song,popularity (Spotify),release date (Spotify)
1,10cc,The Wall Street Shuffle,49.0,2004-01-01
3,2pac,Changes,64.0,1998-01-01
4,3 Doors Down,Be Like That,60.0,2000
5,3 Doors Down,Kryptonite,79.0,2000
6,311,Amber,67.0,2004-06-08


In [12]:
# Display size of dataframe
len(stats)

4773

In [13]:
# Convert release date entries to datetime type and format to show only year
spotify_stats['release date (Spotify)'] = pd.to_datetime(spotify_stats['release date (Spotify)'])
spotify_stats['release date (Spotify)'] = pd.DatetimeIndex(spotify_stats['release date (Spotify)']).year
spotify_stats.head()

,artist,song,popularity (Spotify),release date (Spotify)
1,10cc,The Wall Street Shuffle,49.0,2004
3,2pac,Changes,64.0,1998
4,3 Doors Down,Be Like That,60.0,2000
5,3 Doors Down,Kryptonite,79.0,2000
6,311,Amber,67.0,2004


In [14]:
# Merge full HookTheory data set with last.fm data
combined_lastfm = pd.merge(hook_data, lastfm_stats, how='left', on=['artist','song'])
combined_lastfm.head()

,artist,cp,section,song,type,listener count (last.fm),play count (last.fm),genre (last.fm)
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366.0,1913091.0,rock
1,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366.0,1913091.0,alternative rock
2,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366.0,1913091.0,alternative
3,3 Doors Down,"1,5,6",Verse,Be Like That,3 chord,403366.0,1913091.0,rock
4,3 Doors Down,"1,5,6",Verse,Be Like That,3 chord,403366.0,1913091.0,alternative rock


In [15]:
# Display size of dataframe
len(combined_lastfm)

189221

In [16]:
# Drop duplicate entries from the merged dataframe using values from specific columns as criteria 
combined_lastfm = combined_lastfm.drop_duplicates(subset=['artist','cp','song','genre (last.fm)'])

# Cast listener count and play count data as integers
# combined_lastfm['listener count (last.fm)'] = combined_lastfm['listener count (last.fm)'].astype(int)
# combined_lastfm['play count (last.fm)'] = combined_lastfm['play count (last.fm)'].astype(int)

combined_lastfm.head()

,artist,cp,section,song,type,listener count (last.fm),play count (last.fm),genre (last.fm)
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366.0,1913091.0,rock
1,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366.0,1913091.0,alternative rock
2,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366.0,1913091.0,alternative
6,Adele,"1,5,6",Chorus,Someone Like You,3 chord,1023520.0,10250722.0,soul
7,Adele,"1,5,6",Chorus,Someone Like You,3 chord,1023520.0,10250722.0,piano


In [17]:
# Display size of dataframe
len(combined_lastfm)

164075

In [18]:
# Save resulting dataframe as a csv file
combined_lastfm.to_csv('combined_hook-lastfm.csv')

In [19]:
# Merge full HookTheory data set with Spotify data
combined_spotify = pd.merge(hook_data, spotify_stats, how='left', on=['artist','song'])

# Drop null entries from merged dataframe
combined_spotify = combined_spotify.dropna(how='any')

# Cast popularity and release data as integers
combined_spotify['release date (Spotify)'] = combined_spotify['release date (Spotify)'].astype(int)
combined_spotify['popularity (Spotify)'] = combined_spotify['popularity (Spotify)'].astype(int)

combined_spotify.head()

,artist,cp,section,song,type,popularity (Spotify),release date (Spotify)
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,60,2000
1,3 Doors Down,"1,5,6",Verse,Be Like That,3 chord,60,2000
2,Adele,"1,5,6",Chorus,Someone Like You,3 chord,80,2011
3,Aerosmith,"1,5,6",Verse,Cryin',3 chord,72,1993
5,Arctic Monkeys,"1,5,6",Chorus,Mardy Bum,3 chord,59,2006


In [20]:
# Display size of dataframe
len(combined_spotify)

22518

In [21]:
# Save resulting dataframe as a csv file
combined_spotify.to_csv('combined_hook-spotify.csv')

In [22]:
# Drop duplicate entries from the merged dataframe using values from specific columns as criteria 
unique_spotify = combined_spotify.drop_duplicates(subset=['artist','song'])
unique_spotify.head()

,artist,cp,section,song,type,popularity (Spotify),release date (Spotify)
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,60,2000
2,Adele,"1,5,6",Chorus,Someone Like You,3 chord,80,2011
3,Aerosmith,"1,5,6",Verse,Cryin',3 chord,72,1993
5,Arctic Monkeys,"1,5,6",Chorus,Mardy Bum,3 chord,59,2006
6,Avril Lavigne,"1,5,6",Chorus,Girlfriend,3 chord,74,2007


In [23]:
# Create a truncated dataframe
sub_spotify = unique_spotify[['artist','song','popularity (Spotify)','release date (Spotify)']]
sub_spotify.head()

,artist,song,popularity (Spotify),release date (Spotify)
0,3 Doors Down,Be Like That,60,2000
2,Adele,Someone Like You,80,2011
3,Aerosmith,Cryin',72,1993
5,Arctic Monkeys,Mardy Bum,59,2006
6,Avril Lavigne,Girlfriend,74,2007


In [24]:
combined_all = pd.merge(combined_lastfm, sub_spotify, how='left', on=['artist','song'])
combined_all = combined_all.dropna(how='any')
combined_all['listener count (last.fm)'] = combined_all['listener count (last.fm)'].astype(int)
combined_all['play count (last.fm)'] = combined_all['play count (last.fm)'].astype(int)
combined_all['release date (Spotify)'] = combined_all['release date (Spotify)'].astype(int)
combined_all['popularity (Spotify)'] = combined_all['popularity (Spotify)'].astype(int)
combined_all.head()

,artist,cp,section,song,type,listener count (last.fm),play count (last.fm),genre (last.fm),popularity (Spotify),release date (Spotify)
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366,1913091,rock,60,2000
1,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366,1913091,alternative rock,60,2000
2,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366,1913091,alternative,60,2000
3,Adele,"1,5,6",Chorus,Someone Like You,3 chord,1023520,10250722,soul,80,2011
4,Adele,"1,5,6",Chorus,Someone Like You,3 chord,1023520,10250722,piano,80,2011


In [25]:
# Display size of dataframe
len(combined_all)

93111

In [26]:
# Save resulting dataframe as a csv file
combined_all.to_csv('combined_all.csv')

In [27]:
# Create a dataframe containing only one entry for each song 
combined_unique = combined_all.drop_duplicates(subset=['artist','song'])
combined_unique.head()

,artist,cp,section,song,type,listener count (last.fm),play count (last.fm),genre (last.fm),popularity (Spotify),release date (Spotify)
0,3 Doors Down,"1,5,6",Intro,Be Like That,3 chord,403366,1913091,rock,60,2000
3,Adele,"1,5,6",Chorus,Someone Like You,3 chord,1023520,10250722,soul,80,2011
7,Aerosmith,"1,5,6",Verse,Cryin',3 chord,567063,2970510,rock,72,1993
17,Arctic Monkeys,"1,5,6",Chorus,Mardy Bum,3 chord,1101408,9505520,indie rock,59,2006
22,Avril Lavigne,"1,5,6",Chorus,Girlfriend,3 chord,563039,3833402,pop,74,2007


In [28]:
# Save resulting dataframe as a csv file
combined_unique.to_csv('combined_unique.csv')